In [1]:
%%capture --no-stderr
%pip install --quiet -U kafka-python

In [2]:
from kafka import KafkaProducer, KafkaConsumer
import json

def produce_to_kafka(topic_name, message, bootstrap_servers='localhost:9092'):
    """
    Produces a message to a Kafka topic.

    Args:
        topic_name (str): The name of the Kafka topic.
        message (dict): The message to send (will be serialized as JSON).
        bootstrap_servers (str): Comma-separated list of Kafka broker addresses.
    """
    try:
        producer = KafkaProducer(
            bootstrap_servers=bootstrap_servers,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        producer.send(topic_name, message)
        producer.flush() # Ensure all messages are sent
        print(f"Message sent to topic: {topic_name}")
    except Exception as e:
        print(f"Error producing message: {e}")
    finally:
        if 'producer' in locals():
            producer.close()

def consume_from_kafka(topic_name, bootstrap_servers='localhost:9092', group_id='my-group'):
    """
    Consumes messages from a Kafka topic.

    Args:
        topic_name (str): The name of the Kafka topic.
        bootstrap_servers (str): Comma-separated list of Kafka broker addresses.
        group_id (str): The consumer group ID.
    """
    try:
        consumer = KafkaConsumer(
            topic_name,
            bootstrap_servers=bootstrap_servers,
            group_id=group_id,
            value_deserializer=lambda v: json.loads(v.decode('utf-8'))
        )

        for message in consumer:
            print(f"Received message: {message.value}")

    except Exception as e:
        print(f"Error consuming messages: {e}")
    finally:
        if 'consumer' in locals():
            consumer.close()

In [3]:
topic = "my_new_topic"
servers = '172.200.204.1:9092'

In [4]:
message_to_send = {"key": "value", "number": 123}
produce_to_kafka(topic, message_to_send, servers)

Message sent to topic: my_new_topic


In [5]:
try:
    consume_from_kafka(topic, servers)
except KeyboardInterrupt:
    print("Received KeyboardInterrupt. Quitting.")

Received message: {'key': 'value', 'number': 123}
Received KeyboardInterrupt. Quitting.
